carbon_casefile

In [1]:
function carbon_casefile(file, fuel_dict)
    carbon_casefile(file, fuel_dict)
end

carbon_casefile (generic function with 1 method)

In [2]:
function carbon_casefile(file, fuel_dict)
    # Lookup tables for CO2 and CO2e emissions
    fuel_lookup = Dict( #CO2 GCI
        "ANT" => 0.9095, #Anthracite Coal
        "BIT" => 0.8204, #Bituminous Coal
        "Oil" => 0.7001, #Heavy Oil
        "GAS" => 0.5173,
        "CCGT" => 0.3621, #Gas Combined Cycle
        "ICE" => 0.6030, #Internal Combustion Engine
        "Thermal" => 0.6874, #Thermal Power (General)
        "NUC" => 0.0, #Nuclear Power
        "RE" => 0.0, #Renewable Energy
        "HYD" => 0.0, #Hydropower
        "N/A" => 0.0  # Ensure "N/A" exists
    )

    fuel_lookup_co2e = Dict( #CO2 GCI + (CH4 GCI * 21) + (N2O GCI * 310)
        "ANT" => 0.9143, #Anthracite Coal
        "BIT" => 0.8230, #Bituminous Coal
        "Oil" => 0.7018, #Heavy Oil
        "GAS" => 0.5177,
        "CCGT" => 0.3625, #Gas Combined Cycle
        "ICE" => 0.6049, #Internal Combustion Engine
        "Thermal" => 0.6894, #Thermal Power (General)
        "NUC" => 0.0, #Nuclear Power
        "RE" => 0.0, #Renewable Energy
        "HYD" => 0.0, #Hydropower
        "N/A" => 0.0  # Ensure "N/A" exists
    )

    for (bus_id, info) in fuel_dict
        # Check if both "type" and "emissions" exist before accessing them
        if haskey(info, "type") && haskey(info, "emissions")
            # Find generator IDs that match the given bus number
            matching_gen_ids = [string(gen_id) for (gen_id, gen_data) in file["gen"] if get(gen_data, "gen_bus", -1) == bus_id]

            # Determine the correct lookup table
            lookup_table = info["emissions"] == "CO2e" ? fuel_lookup_co2e : fuel_lookup

            # Assign GCI values to all matching generators
            for gen_id in matching_gen_ids
                file["gen"][gen_id]["GCI"] = get(lookup_table, info["type"], 0.0)  # Default to 0.0 if type not found
            end
        end
    end
end


carbon_casefile (generic function with 1 method)

In [3]:
function fuel_dict_generation(file)
    fuel_dict_generation(file)
end

fuel_dict_generation (generic function with 1 method)

In [4]:
function fuel_dict_generation(file)
    fuel_dict = Dict{Int, Dict{String, String}}()  # Initialize an empty dictionary

    for (_, gen_data) in file["gen"]  # Iterate over generator data
        gen_bus = get(gen_data, "gen_bus", nothing)  # Safely get "gen_bus"
        if gen_bus !== nothing && !haskey(fuel_dict, gen_bus)  # Ensure "gen_bus" exists and is not already added
            fuel_dict[gen_bus] = Dict("type" => "N/A", "emissions" => "CO2")  # Assign default values
        end
    end
    return fuel_dict
end


fuel_dict_generation (generic function with 1 method)

Example

In [5]:
using PowerModels
using HiGHS

In [6]:
# Load the Powermodels.jl network (assumed to be in MATPOWER format)
net = PowerModels.parse_file("case118.m")

[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 1 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 1 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 54 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 54 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 101 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 101 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 41 from -360.0 to -60.0 deg

Dict{String, Any} with 13 entries:
  "bus"            => Dict{String, Any}("32"=>Dict{String, Any}("zone"=>1, "bus…
  "source_type"    => "matpower"
  "name"           => "case118"
  "dcline"         => Dict{String, Any}()
  "source_version" => "2"
  "gen"            => Dict{String, Any}("32"=>Dict{String, Any}("ncost"=>3, "qc…
  "branch"         => Dict{String, Any}("1"=>Dict{String, Any}("br_r"=>0.0303, …
  "storage"        => Dict{String, Any}()
  "switch"         => Dict{String, Any}()
  "baseMVA"        => 100
  "per_unit"       => true
  "shunt"          => Dict{String, Any}("4"=>Dict{String, Any}("source_id"=>Any…
  "load"           => Dict{String, Any}("32"=>Dict{String, Any}("source_id"=>An…

In [7]:
# Generate the fuel dictionary
fuel_dict = fuel_dict_generation(net)

Dict{Int64, Dict{String, String}} with 54 entries:
  56  => Dict("type"=>"N/A", "emissions"=>"N/A")
  55  => Dict("type"=>"N/A", "emissions"=>"N/A")
  110 => Dict("type"=>"N/A", "emissions"=>"N/A")
  32  => Dict("type"=>"N/A", "emissions"=>"N/A")
  6   => Dict("type"=>"N/A", "emissions"=>"N/A")
  73  => Dict("type"=>"N/A", "emissions"=>"N/A")
  112 => Dict("type"=>"N/A", "emissions"=>"N/A")
  90  => Dict("type"=>"N/A", "emissions"=>"N/A")
  4   => Dict("type"=>"N/A", "emissions"=>"N/A")
  54  => Dict("type"=>"N/A", "emissions"=>"N/A")
  104 => Dict("type"=>"N/A", "emissions"=>"N/A")
  91  => Dict("type"=>"N/A", "emissions"=>"N/A")
  62  => Dict("type"=>"N/A", "emissions"=>"N/A")
  12  => Dict("type"=>"N/A", "emissions"=>"N/A")
  111 => Dict("type"=>"N/A", "emissions"=>"N/A")
  92  => Dict("type"=>"N/A", "emissions"=>"N/A")
  69  => Dict("type"=>"N/A", "emissions"=>"N/A")
  36  => Dict("type"=>"N/A", "emissions"=>"N/A")
  85  => Dict("type"=>"N/A", "emissions"=>"N/A")
  77  => Dict("typ

In [8]:
# Assign fuel types and emissions to specific generator buses
fuel_dict[10] = Dict("type" => "GAS", "emissions" => "CO2e")
fuel_dict[12] = Dict("type" => "ANT", "emissions" => "CO2")
fuel_dict[110] = Dict("type" => "BIT", "emissions" => "CO2")

Dict{String, String} with 2 entries:
  "type"      => "BIT"
  "emissions" => "CO2"

In [9]:
# Apply carbon constraints to the PowerModels network
carbon_casefile(net, fuel_dict)

In [10]:
net["gen"]

Dict{String, Any} with 54 entries:
  "32" => Dict{String, Any}("ncost"=>3, "qc1max"=>0.0, "pg"=>0.0, "model"=>2, "…
  "29" => Dict{String, Any}("ncost"=>3, "qc1max"=>0.0, "pg"=>3.92, "model"=>2, …
  "1"  => Dict{String, Any}("ncost"=>3, "qc1max"=>0.0, "pg"=>0.0, "model"=>2, "…
  "54" => Dict{String, Any}("ncost"=>3, "qc1max"=>0.0, "pg"=>0.0, "model"=>2, "…
  "2"  => Dict{String, Any}("ncost"=>3, "qc1max"=>0.0, "pg"=>0.0, "model"=>2, "…
  "41" => Dict{String, Any}("ncost"=>3, "qc1max"=>0.0, "pg"=>0.0, "model"=>2, "…
  "51" => Dict{String, Any}("ncost"=>3, "qc1max"=>0.0, "pg"=>0.36, "model"=>2, …
  "53" => Dict{String, Any}("ncost"=>3, "qc1max"=>0.0, "pg"=>0.0, "model"=>2, "…
  "27" => Dict{String, Any}("ncost"=>3, "qc1max"=>0.0, "pg"=>0.0, "model"=>2, "…
  "42" => Dict{String, Any}("ncost"=>3, "qc1max"=>0.0, "pg"=>0.0, "model"=>2, "…
  "33" => Dict{String, Any}("ncost"=>3, "qc1max"=>0.0, "pg"=>0.0, "model"=>2, "…
  "28" => Dict{String, Any}("ncost"=>3, "qc1max"=>0.0, "pg"=>3.91, "model"

In [11]:
# Run DCOPF
result=solve_dc_opf(net, HiGHS.Optimizer)

Running HiGHS 1.7.0 (git hash: 50670fd4c): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 2e+02]
  Cost   [2e+03, 4e+03]
  Bound  [1e+00, 8e+00]
  RHS    [2e-02, 3e+00]
Iteration, Runtime, ObjVal, NullspaceDim
0, 0.009147, 198219.396904, 11
100, 0.014219, 150784.114014, 6
200, 0.018504, 150784.114014, 6
300, 0.022910, 131715.078542, 18
356, 0.024950, 125947.881432, 18
Model   status      : Optimal
Simplex   iterations: 273
Objective value     :  1.2594788142e+05
HiGHS run time      :          0.02


Dict{String, Any} with 8 entries:
  "solve_time"         => 0.0249498
  "optimizer"          => "HiGHS"
  "termination_status" => OPTIMAL
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 1.25948e5
  "solution"           => Dict{String, Any}("baseMVA"=>100, "branch"=>Dict{Stri…
  "objective_lb"       => 0.0

In [12]:
# Calculate carbon emissions for each generator
for (bus_number, gen_data) in result["solution"]["gen"]
    # Retrieve power generation value (pg)
    pg = gen_data["pg"]
    # Retrieve GCI value from net["gen"]
    if haskey(net["gen"], bus_number) && haskey(net["gen"][bus_number], "GCI")
        GCI = net["gen"][bus_number]["GCI"]  # Get Generation Carbon Intensity
        carbon_emission = pg * GCI       # Compute Carbon Emission
    else
        carbon_emission = NaN  # If GCI is missing, set emission as NaN
    end

    # Add carbon emission to the generator data
    gen_data["carbon_emission"] = carbon_emission
end

In [13]:
for i in 1:54
    println(result["solution"]["gen"][string(i)]["carbon_emission"])
end

0.0
0.0
0.0
0.0
2.257590193330223
0.7491625505349903
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
